In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import pairwise_distances

import utils.sulfur.constant as const
from utils.CaImageCSV import CaImageCSV

In [2]:
def droped_unique_cells(cells1, cells2):
    droped_cells = np.array([])
    droped_cells = np.append(droped_cells, cells1)
    droped_cells = np.append(droped_cells, cells2)
    droped_cells = np.unique(droped_cells)
    return droped_cells

def all_nan_cells(matrix):
    nan_cells = np.array([])
    for cell_name, items in matrix.iteritems():
        if items.isnull().all():
            nan_cells = np.append(nan_cells, cell_name)
            
    return nan_cells

def splitted_context_name(context_name, index):
    return '%s_%d' % (context_name, index)

def build_matrix_index(context_name, cells):
    index_df = pd.DataFrame(data=([context_name] * len(cells)))
    index_df.columns = ['context']
    cells = cells.reset_index(drop=True)
    index_df['cell_name'] = cells
    return pd.MultiIndex.from_frame(index_df, names=['context_name', 'cell_name'])

def split_matrix(matrix, context_name):
    matrix_1 = matrix.iloc[0:400, :]
    matrix_2 = matrix.iloc[900:1300, :]
    matrix_3 = matrix.iloc[1800:2200, :]
    
    matrix_1.rename(lambda x: splitted_context_name(x, 1), inplace=True)
    matrix_2.rename(lambda x: splitted_context_name(x, 2), inplace=True)
    matrix_3.rename(lambda x: splitted_context_name(x, 3), inplace=True)
    
    return matrix_1, matrix_2, matrix_3

def split_engram_matrix(origin, context_name):
    matrix = origin.filtered_by_context(context_name).astype(float).copy(deep=True)
    matrix = origin.optimize_std(matrix)
    matrix.reset_index(level=1, drop=True, inplace=True)
    
    engram_matrix = matrix.loc[:, origin.engram_cells()]
    non_engram_matrix = matrix.loc[:, csv.non_engram_cells()]
    
    return engram_matrix, non_engram_matrix

In [3]:
index = const.ANIMAL_NAMES
column_df = pd.DataFrame()
column_df['cell type'] = ['engram', 'engram', 'engram', 'engram', 'engram', 'engram', 'engram', 'non-engram', 'non-engram', 'non-engram', 'non-engram', 'non-engram', 'non-engram', 'non-engram']
column_df['context'] = ['A1preES_2', 'A1postES_1', 'A1postES_2', 'A2_1', 'A2_2', 'A3_1', 'A3_2'] * 2
columns = pd.MultiIndex.from_frame(column_df)
across_session_df = pd.DataFrame(0.0, columns=columns, index=index)
inner_session_df = pd.DataFrame(0.0, columns=columns, index=index)
for animal_name in const.ANIMAL_NAMES:
    print(animal_name)
    file_path = './resources/%s_Longitudinal_Traces.csv' % animal_name
    csv = CaImageCSV(file_path)
    
    # A1preES
    A1preES_engram, A1preES_non_engram = split_engram_matrix(csv, 'A1preES')
    
    A1preES_nan_engram_cells = all_nan_cells(A1preES_engram)
    A1preES_nan_non_engram_cells = all_nan_cells(A1preES_non_engram)
    
    # A1postES
    A1postES_engram, A1postES_non_engram = split_engram_matrix(csv, 'A1postES')
    
    A1postES_nan_engram_cells = all_nan_cells(A1postES_engram)
    A1postES_nan_non_engram_cells = all_nan_cells(A1postES_non_engram)
    
    # A2_1, A2_2
    A2_1_engram, A2_1_non_engram = split_engram_matrix(csv, 'A2_1')
    
    A2_1_nan_engram_cells = all_nan_cells(A2_1_engram)
    A2_1_nan_non_engram_cells = all_nan_cells(A2_1_non_engram)
    
    A2_2_engram, A2_2_non_engram = split_engram_matrix(csv, 'A2_2')
    
    A2_2_nan_engram_cells = all_nan_cells(A2_2_engram)
    A2_2_nan_non_engram_cells = all_nan_cells(A2_2_non_engram)
    
    # A3_1, A3_2
    A3_1_engram, A3_1_non_engram = split_engram_matrix(csv, 'A3_1')
    
    A3_1_nan_engram_cells = all_nan_cells(A3_1_engram)
    A3_1_nan_non_engram_cells = all_nan_cells(A3_1_non_engram)
    
    A3_2_engram, A3_2_non_engram = split_engram_matrix(csv, 'A3_2')
    
    A3_2_nan_engram_cells = all_nan_cells(A3_2_engram)
    A3_2_nan_non_engram_cells = all_nan_cells(A3_2_non_engram)
    
    # fetch dropped cells
    dropped_engram_cells = droped_unique_cells(A1preES_nan_engram_cells, A1postES_nan_engram_cells)
    dropped_engram_cells = droped_unique_cells(dropped_engram_cells, A2_1_nan_engram_cells)
    dropped_engram_cells = droped_unique_cells(dropped_engram_cells, A2_2_nan_engram_cells)
    dropped_engram_cells = droped_unique_cells(dropped_engram_cells, A3_1_nan_engram_cells)
    dropped_engram_cells = droped_unique_cells(dropped_engram_cells, A3_2_nan_engram_cells)
    
    dropped_non_engram_cells = droped_unique_cells(A1preES_nan_non_engram_cells, A1postES_nan_non_engram_cells)
    dropped_non_engram_cells = droped_unique_cells(dropped_non_engram_cells, A2_1_nan_non_engram_cells)
    dropped_non_engram_cells = droped_unique_cells(dropped_non_engram_cells, A2_2_nan_non_engram_cells)
    dropped_non_engram_cells = droped_unique_cells(dropped_non_engram_cells, A3_1_nan_non_engram_cells)
    dropped_non_engram_cells = droped_unique_cells(dropped_non_engram_cells, A3_2_nan_non_engram_cells)
    
    # A1preES
    A1preES_engram.drop(columns=dropped_engram_cells, inplace=True)
    A1preES_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    #A1preES_engram_1 = A1preES_engram.iloc[0:400, :]
    #A1preES_engram_1.rename(lambda x: splitted_context_name(x, 1), inplace=True)
    
    A1preES_engram_2 = A1preES_engram.iloc[2600:3000, :]
    A1preES_engram_2.rename(lambda x: splitted_context_name(x, 2), inplace=True)
    
    #A1preES_non_engram_1 = A1preES_non_engram.iloc[0:400, :]
    #A1preES_non_engram_1.rename(lambda x: splitted_context_name(x, 1), inplace=True)
    
    A1preES_non_engram_2 = A1preES_non_engram.iloc[2600:3000, :]
    A1preES_non_engram_2.rename(lambda x: splitted_context_name(x, 2), inplace=True)
    
    # A1postES
    A1postES_engram.drop(columns=dropped_engram_cells, inplace=True)
    A1postES_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    A1postES_engram_1 = A1postES_engram.iloc[0:400, :]
    A1postES_engram_1.rename(lambda x: splitted_context_name(x, 1), inplace=True)
    
    A1postES_engram_2 = A1postES_engram.iloc[1800:2200, :]
    A1postES_engram_2.rename(lambda x: splitted_context_name(x, 2), inplace=True)
    
    A1postES_non_engram_1 = A1postES_non_engram.iloc[0:400, :]
    A1postES_non_engram_1.rename(lambda x: splitted_context_name(x, 1), inplace=True)
    
    A1postES_non_engram_2 = A1postES_non_engram.iloc[1800:2200, :]
    A1postES_non_engram_2.rename(lambda x: splitted_context_name(x, 2), inplace=True)
    
    # A2_1, A2_2
    A2_1_engram.drop(columns=dropped_engram_cells, inplace=True)
    A2_1_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    A2_2_engram.drop(columns=dropped_engram_cells, inplace=True)
    A2_2_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    A2_1_engram_start = A2_1_engram.iloc[0:400, :]
    A2_2_engram_end = A2_2_engram.iloc[2600:3000, :]
    
    A2_1_non_engram_start = A2_1_non_engram.iloc[0:400, :]
    A2_2_non_engram_end = A2_2_non_engram.iloc[2600:3000, :]
    
    # A3_1, A3_2
    A3_1_engram.drop(columns=dropped_engram_cells, inplace=True)
    A3_1_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    A3_2_engram.drop(columns=dropped_engram_cells, inplace=True)
    A3_2_non_engram.drop(columns=dropped_non_engram_cells, inplace=True)
    
    A3_1_engram_start = A3_1_engram.iloc[0:400, :]
    A3_2_engram_end = A3_2_engram.iloc[2600:3000, :]
    
    A3_1_non_engram_start = A3_1_non_engram.iloc[0:400, :]
    A3_2_non_engram_end = A3_2_non_engram.iloc[2600:3000, :]
    
    across_session_df.loc[animal_name, ('engram', 'A1preES_2')]  = np.mean(pairwise_distances(A1preES_engram_2, A1preES_engram_2))
    across_session_df.loc[animal_name, ('engram', 'A1postES_1')] = np.mean(pairwise_distances(A1preES_engram_2, A1postES_engram_1))
    across_session_df.loc[animal_name, ('engram', 'A1postES_2')] = np.mean(pairwise_distances(A1preES_engram_2, A1postES_engram_2))
    across_session_df.loc[animal_name, ('engram', 'A2_1')]       = np.mean(pairwise_distances(A1preES_engram_2, A2_1_engram_start))
    across_session_df.loc[animal_name, ('engram', 'A2_2')]       = np.mean(pairwise_distances(A1preES_engram_2, A2_2_engram_end))
    across_session_df.loc[animal_name, ('engram', 'A3_1')]       = np.mean(pairwise_distances(A1preES_engram_2, A3_1_engram_start))
    across_session_df.loc[animal_name, ('engram', 'A3_2')]       = np.mean(pairwise_distances(A1preES_engram_2, A3_2_engram_end))
    
    across_session_df.loc[animal_name, ('non-engram', 'A1preES_2')]  = np.mean(pairwise_distances(A1preES_non_engram_2, A1preES_non_engram_2))
    across_session_df.loc[animal_name, ('non-engram', 'A1postES_1')] = np.mean(pairwise_distances(A1preES_non_engram_2, A1postES_non_engram_1))
    across_session_df.loc[animal_name, ('non-engram', 'A1postES_2')] = np.mean(pairwise_distances(A1preES_non_engram_2, A1postES_non_engram_2))
    across_session_df.loc[animal_name, ('non-engram', 'A2_1')]       = np.mean(pairwise_distances(A1preES_non_engram_2, A2_1_non_engram_start))
    across_session_df.loc[animal_name, ('non-engram', 'A2_2')]       = np.mean(pairwise_distances(A1preES_non_engram_2, A2_2_non_engram_end))
    across_session_df.loc[animal_name, ('non-engram', 'A3_1')]       = np.mean(pairwise_distances(A1preES_non_engram_2, A3_1_non_engram_start))
    across_session_df.loc[animal_name, ('non-engram', 'A3_2')]       = np.mean(pairwise_distances(A1preES_non_engram_2, A3_2_non_engram_end))
    
    #inner_session_df.loc[animal_name, ('engram', 'A1preES_1')]  = np.mean(pairwise_distances(A1preES_engram_1, A1preES_engram_1, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A1preES_2')]  = np.mean(pairwise_distances(A1preES_engram_2, A1preES_engram_2, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A1postES_1')] = np.mean(pairwise_distances(A1postES_engram_1, A1postES_engram_1, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A1postES_2')] = np.mean(pairwise_distances(A1postES_engram_2, A1postES_engram_2, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A2_1')]       = np.mean(pairwise_distances(A2_1_engram_start, A2_1_engram_start, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A2_2')]       = np.mean(pairwise_distances(A2_2_engram_end, A2_2_engram_end, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A3_1')]       = np.mean(pairwise_distances(A3_1_engram_start, A3_1_engram_start, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('engram', 'A3_2')]       = np.mean(pairwise_distances(A3_2_engram_end, A3_2_engram_end, metric='mahalanobis'))
    #
    #inner_session_df.loc[animal_name, ('non-engram', 'A1preES_1')]  = np.mean(pairwise_distances(A1preES_non_engram_1, A1preES_non_engram_1, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A1preES_2')]  = np.mean(pairwise_distances(A1preES_non_engram_2, A1preES_non_engram_2, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A1postES_1')] = np.mean(pairwise_distances(A1postES_non_engram_1, A1postES_non_engram_1, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A1postES_2')] = np.mean(pairwise_distances(A1postES_non_engram_2, A1postES_non_engram_2, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A2_1')]       = np.mean(pairwise_distances(A2_1_non_engram_start, A2_1_non_engram_start, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A2_2')]       = np.mean(pairwise_distances(A2_2_non_engram_end, A2_2_non_engram_end, metric='mahalanobis'))
    #inner_session_df.loc[animal_name, ('non-engram', 'A3_1')]       = np.mean(pairwise_distances(A3_1_non_engram_start, A3_1_non_engram_start, metric='mahalanobis'))
    #print(A3_2_non_engram_end)
    #inner_session_df.loc[animal_name, ('non-engram', 'A3_2')]       = np.mean(pairwise_distances(A3_2_non_engram_end, A3_2_non_engram_end, metric='mahalanobis'))


ID181106CreA


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

ID181106CreB


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

ID181106CreC


...

ID181106CreI


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

ID181106CreK


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

LinAlgError: Singular matrix

In [ ]:
#inner_session_df.to_csv('./resources/pairwise_distance/extinction/inner_session.csv')

In [ ]:
across_session_df.to_csv('./resources/pairwise_distance/extinction/across_session_by_mahalanobis_ver_a1_2.csv')

In [ ]:
calc_across_session_df = across_session_df.copy(deep=True)
for index, rows in calc_across_session_df.iloc[:, 0:7].iterrows():
    first_value = rows[0]
    rows -= first_value
for index, rows in calc_across_session_df.iloc[:, 7:].iterrows():
    first_value = rows[0]
    rows -= first_value
calc_across_session_df

In [ ]:
calc_across_session_df.to_csv('./resources/pairwise_distance/extinction/calculated_across_session_by_mahalanobis_ver_a1_2.csv')

In [ ]:
engram_distances = calc_across_session_df.iloc[:, 0:7]
non_engram_distances = calc_across_session_df.iloc[:, 7:]

x = engram_distances.columns.get_level_values('context').to_numpy()
engram_y, engram_err = engram_distances.mean(), engram_distances.sem()
non_engram_y, non_engram_err = non_engram_distances.mean(), non_engram_distances.sem()

plt.figure(figsize=(10, 7), dpi=200)
plt.errorbar(x, engram_y, yerr=engram_err, capsize=2, c='firebrick')
plt.plot(x, engram_y, marker='s', markersize=10, label='engram cell', c='firebrick', zorder=12)
plt.errorbar(x, non_engram_y, yerr=non_engram_err, capsize=2, c='navy')
plt.plot(x, non_engram_y, marker='o', markersize=10, label='non-engram cell', c='navy', zorder=12)

for index, rows in engram_distances.iterrows():
    plt.plot(x, rows, marker='s', markersize=10, c='firebrick', alpha=0.1)
    
for index, rows in non_engram_distances.iterrows():
    plt.plot(x, rows, marker='o', markersize=10, c='navy', alpha=0.1)

plt.xticks(rotation=45)
plt.xlabel('session')
plt.ylabel('Population Vector Distance')
plt.legend()
plt.show()